# <center>**THUẬT TOÁN K-NEAREST NEIGHBORS (KNN)**<center>

## **I. Câu 1: Toy Example – Phân loại loài hoa Iris (UCI Iris Dataset)**
## **II. Câu 2: Nhận dạng ký tự A-Z (UCI Letter Recognition Dataset)**

---

In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import operator
import random
import math

---
# **I. CÂU 1: PHÂN LOẠI LOÀI HOA IRIS (Toy Example)**

### **Giới thiệu**
- **Tập dữ liệu:** UCI Iris (150 mẫu, 4 đặc trưng, 3 lớp: `Iris-setosa`, `Iris-versicolor`, `Iris-virginica`)
- **Mục tiêu:** Demo thuật toán KNN từ đầu (from scratch)
- **Tham khảo:**
  - https://alphacoder.xyz/k-nearest-neighbors
  - https://archive.ics.uci.edu/ml/datasets/iris

### **Hướng dẫn cài đặt & chạy demo**
```bash
# 1. Cài đặt môi trường
pip install numpy pandas jupyter

# 2. Chạy Jupyter
jupyter notebook

In [2]:
def euclidean_distance(row1, row2):
    return math.sqrt(sum((a - b) ** 2 for a, b in zip(row1[:-1], row2[:-1])))

def get_neighbors(training_set, test_row, k):
    distances = [(train_row, euclidean_distance(test_row, train_row)) for train_row in training_set]
    distances.sort(key=operator.itemgetter(1))
    return [distances[i][0] for i in range(k)]

def get_prediction(neighbors):
    votes = Counter(row[-1] for row in neighbors)
    return votes.most_common(1)[0][0]

def get_accuracy(test_set, predictions):
    correct = sum(1 for i, row in enumerate(test_set) if row[-1] == predictions[i])
    return (correct / len(test_set)) * 100.0

def get_weighted_prediction(nearest_k_with_dist):
    votes = {}
    for item in nearest_k_with_dist:
        label = item[0][-1]
        dist = item[1]
        weight = 1.0 / (dist ** 2) if dist > 0 else 1e10
        votes[label] = votes.get(label, 0) + weight
    return max(votes.items(), key=operator.itemgetter(1))[0]

### **Tải dữ liệu Iris**

In [3]:
def load_iris_dataset(file_name="iris.data", training_ratio=0.67):
    df = pd.read_csv(file_name, header=None)
    df.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
    dataset = df.values.tolist()
    for row in dataset:
        row[:4] = [float(x) for x in row[:4]]
    random.seed(42)
    random.shuffle(dataset)
    split = int(len(dataset) * training_ratio)
    return dataset[:split], dataset[split:]

training_set_iris, test_set_iris = load_iris_dataset()
print(f"Đã tải dữ liệu Iris: {len(training_set_iris)} mẫu huấn luyện, {len(test_set_iris)} mẫu kiểm tra.")

Đã tải dữ liệu Iris: 100 mẫu huấn luyện, 50 mẫu kiểm tra.


### **Demo KNN cơ bản (K=3) + Ví dụ đúng/sai**

In [4]:
print("\nBắt đầu phân loại Iris với K = 3...\n")

K = 3
predictions = []
correct_examples = []
wrong_examples = []

for idx, test_sample in enumerate(test_set_iris):
    neighbors = get_neighbors(training_set_iris, test_sample, K)
    result = get_prediction(neighbors)
    predictions.append(result)
    
    neighbor_labels = [n[-1] for n in neighbors]
    
    if result == test_sample[-1]:
        if len(correct_examples) < 3:
            correct_examples.append((test_sample[-1], result, neighbor_labels))
    else:
        if len(wrong_examples) < 3:
            wrong_examples.append((test_sample[-1], result, neighbor_labels, test_sample[:4]))

accuracy = get_accuracy(test_set_iris, predictions)
print(f"==================================================\n--- KẾT QUẢ PHÂN LOẠI KNN TRÊN IRIS ---\nK-Value: {K}\nĐộ chính xác: {accuracy:.2f}%\n==================================================\n")

print("*** VÍ DỤ DỰ ĐOÁN ĐÚNG (CORRECT) ***")
for i, ex in enumerate(correct_examples, 1):
    print(f"✅ Ví dụ {i}: Nhãn THỰC TẾ: {ex[0]}, DỰ ĐOÁN: {ex[1]}")
    print(f"   * {K} láng giềng: {ex[2]}")
    print("------------------------------")

print("\n*** VÍ DỤ DỰ ĐOÁN SAI (WRONG) ***")
for i, ex in enumerate(wrong_examples, 1):
    print(f"❌ Ví dụ {i}: Nhãn THỰC TẾ: {ex[0]}, DỰ ĐOÁN: {ex[1]}")
    print(f"   * {K} láng giềng: {ex[2]}")
    print(f"   * 4 đặc trưng: {ex[3]}")
    print("------------------------------")


Bắt đầu phân loại Iris với K = 3...

--- KẾT QUẢ PHÂN LOẠI KNN TRÊN IRIS ---
K-Value: 3
Độ chính xác: 98.00%

*** VÍ DỤ DỰ ĐOÁN ĐÚNG (CORRECT) ***
✅ Ví dụ 1: Nhãn THỰC TẾ: Iris-virginica, DỰ ĐOÁN: Iris-virginica
   * 3 láng giềng: ['Iris-virginica', 'Iris-virginica', 'Iris-versicolor']
------------------------------
✅ Ví dụ 2: Nhãn THỰC TẾ: Iris-setosa, DỰ ĐOÁN: Iris-setosa
   * 3 láng giềng: ['Iris-setosa', 'Iris-setosa', 'Iris-setosa']
------------------------------
✅ Ví dụ 3: Nhãn THỰC TẾ: Iris-versicolor, DỰ ĐOÁN: Iris-versicolor
   * 3 láng giềng: ['Iris-versicolor', 'Iris-versicolor', 'Iris-virginica']
------------------------------

*** VÍ DỤ DỰ ĐOÁN SAI (WRONG) ***
❌ Ví dụ 1: Nhãn THỰC TẾ: Iris-versicolor, DỰ ĐOÁN: Iris-virginica
   * 3 láng giềng: ['Iris-virginica', 'Iris-virginica', 'Iris-virginica']
   * 4 đặc trưng: [5.9, 3.2, 4.8, 1.8]
------------------------------


### **Cải thiện: Tối ưu K + Weighted KNN**

In [5]:
def main_iris_improved():
    training_set, test_set = load_iris_dataset(training_ratio=0.67)
    
    print("\n" + "="*50)
    print("--- 1. TỐI ƯU HÓA GIÁ TRỊ K CHO IRIS ---")
    best_k = 1
    max_accuracy = 0
    
    for K_VALUE in range(1, 20, 2):
        predictions = [get_prediction(get_neighbors(training_set, row, K_VALUE)) for row in test_set]
        accuracy = get_accuracy(test_set, predictions)
        print(f"  * K={K_VALUE:2d}: Độ chính xác = {accuracy:5.2f}%")
        if accuracy > max_accuracy:
            max_accuracy = accuracy
            best_k = K_VALUE
    
    print(f"\n=> K tốt nhất (Majority Vote): K={best_k} → {max_accuracy:.2f}%\n")
    
    print("="*50)
    print("--- 2. WEIGHTED KNN (trọng số 1/d²) ---")
    predictions_weighted = []
    for test_sample in test_set:
        distances = [(train, euclidean_distance(test_sample, train)) for train in training_set]
        distances.sort(key=operator.itemgetter(1))
        nearest_k = distances[:best_k]
        result = get_weighted_prediction(nearest_k)
        predictions_weighted.append(result)
    
    accuracy_weighted = get_accuracy(test_set, predictions_weighted)
    print(f"  * K={best_k} + Weighted Vote → Độ chính xác = {accuracy_weighted:.2f}%")
    print("="*50)

main_iris_improved()


--- 1. TỐI ƯU HÓA GIÁ TRỊ K CHO IRIS ---
  * K= 1: Độ chính xác = 98.00%
  * K= 3: Độ chính xác = 98.00%
  * K= 5: Độ chính xác = 98.00%
  * K= 7: Độ chính xác = 98.00%
  * K= 9: Độ chính xác = 98.00%
  * K=11: Độ chính xác = 98.00%
  * K=13: Độ chính xác = 98.00%
  * K=15: Độ chính xác = 98.00%
  * K=17: Độ chính xác = 98.00%
  * K=19: Độ chính xác = 98.00%

=> K tốt nhất (Majority Vote): K=1 → 98.00%

--- 2. WEIGHTED KNN (trọng số 1/d²) ---
  * K=1 + Weighted Vote → Độ chính xác = 98.00%


---
# **II. CÂU 2: NHẬN DẠNG KÝ TỰ A-Z (UCI Letter Recognition)**

### **Giới thiệu**
- **Tập dữ liệu:** 20,000 mẫu, 26 lớp (A-Z), 16 đặc trưng  
- **Phân chia:** 16,000 huấn luyện, 4,000 kiểm tra  
- **Thư viện:** `scikit-learn` (KNeighborsClassifier)  
- **Ưu điểm:** Tốc độ cao, hỗ trợ weighted vote  
- **Dữ liệu:** https://archive.ics.uci.edu/ml/datasets/letter+recognition

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

### **Tải dữ liệu letter recognition**

In [7]:
def load_letter_sklearn(file_name="letter-recognition.data"):
    df = pd.read_csv(file_name, header=None)
    X = df.iloc[:, 1:].values.astype(float)   # 16 features
    y = df.iloc[:, 0].values                  # labels A-Z
    return X, y

print("Đang tải dữ liệu Letter Recognition (20,000 mẫu)...")
X, y = load_letter_sklearn()

Đang tải dữ liệu Letter Recognition (20,000 mẫu)...


### **Chia train/test (seed cố định)**

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=16000, test_size=4000, random_state=42, stratify=y
)
print(f"Hoàn tất: {len(X_train)} train, {len(X_test)} test.\n")

Hoàn tất: 16000 train, 4000 test.



### **Demo KNN cơ bản (K=5) + Ví dụ đúng/sai**

In [9]:
print("Bắt đầu demo KNN (K=5) với sklearn...\n")

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred) * 100
print(f"{'='*55}")
print(f"--- KẾT QUẢ KNN (K=5) ---")
print(f"Độ chính xác: {accuracy:.2f}%")
print(f"{'='*55}\n")

# Lấy 3 ví dụ đúng + 3 ví dụ sai
correct_idx = np.where(y_test == y_pred)[0]
wrong_idx = np.where(y_test != y_pred)[0]

print("*** VÍ DỤ DỰ ĐOÁN ĐÚNG (CORRECT) ***")
for i, idx in enumerate(correct_idx[:3], 1):
    neighbors = knn.kneighbors([X_test[idx]], n_neighbors=5, return_distance=False)[0]
    neighbor_labels = [y_train[n] for n in neighbors]
    print(f"Positive Ví dụ {i}: THỰC TẾ: {y_test[idx]}, DỰ ĐOÁN: {y_pred[idx]}")
    print(f"   * 5 láng giềng: {neighbor_labels}")
    print("------------------------------")

print("\n*** VÍ DỤ DỰ ĐOÁN SAI (WRONG) ***")
for i, idx in enumerate(wrong_idx[:3], 1):
    neighbors = knn.kneighbors([X_test[idx]], n_neighbors=5, return_distance=False)[0]
    neighbor_labels = [y_train[n] for n in neighbors]
    print(f"Negative Ví dụ {i}: THỰC TẾ: {y_test[idx]}, DỰ ĐOÁN: {y_pred[idx]}")
    print(f"   * 5 láng giềng: {neighbor_labels}")
    print(f"   * 16 đặc trưng: {X_test[idx].astype(int).tolist()}")
    print("------------------------------")

Bắt đầu demo KNN (K=5) với sklearn...

--- KẾT QUẢ KNN (K=5) ---
Độ chính xác: 95.33%

*** VÍ DỤ DỰ ĐOÁN ĐÚNG (CORRECT) ***
Positive Ví dụ 1: THỰC TẾ: R, DỰ ĐOÁN: R
   * 5 láng giềng: ['R', 'R', 'R', 'R', 'R']
------------------------------
Positive Ví dụ 2: THỰC TẾ: M, DỰ ĐOÁN: M
   * 5 láng giềng: ['M', 'M', 'M', 'M', 'M']
------------------------------
Positive Ví dụ 3: THỰC TẾ: B, DỰ ĐOÁN: B
   * 5 láng giềng: ['B', 'B', 'B', 'B', 'B']
------------------------------

*** VÍ DỤ DỰ ĐOÁN SAI (WRONG) ***
Negative Ví dụ 1: THỰC TẾ: V, DỰ ĐOÁN: U
   * 5 láng giềng: ['C', 'V', 'O', 'U', 'U']
   * 16 đặc trưng: [4, 9, 6, 7, 7, 7, 8, 3, 2, 7, 8, 8, 8, 9, 6, 7]
------------------------------
Negative Ví dụ 2: THỰC TẾ: N, DỰ ĐOÁN: M
   * 5 láng giềng: ['V', 'N', 'M', 'M', 'M']
   * 16 đặc trưng: [7, 9, 9, 8, 10, 8, 7, 6, 5, 7, 6, 7, 9, 9, 8, 0]
------------------------------
Negative Ví dụ 3: THỰC TẾ: O, DỰ ĐOÁN: C
   * 5 láng giềng: ['O', 'O', 'C', 'C', 'H']
   * 16 đặc trưng: [8, 15, 5, 8, 

### **Cải thiện: Tối ưu K + Weighted KNN**

In [10]:
print("\nTối ưu hóa K và thử Weighted KNN...\n")

best_k = 1
best_acc = 0
accuracies = {}

for k in range(1, 12, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    acc = accuracy_score(y_test, knn.predict(X_test)) * 100
    accuracies[k] = acc
    print(f"  * K={k:2d}: {acc:6.2f}%")
    if acc > best_acc:
        best_acc = acc
        best_k = k

print(f"\n=> K tốt nhất (uniform): K={best_k} → {best_acc:.2f}%")

# Weighted KNN
knn_weighted = KNeighborsClassifier(n_neighbors=best_k, weights='distance')
knn_weighted.fit(X_train, y_train)
acc_weighted = accuracy_score(y_test, knn_weighted.predict(X_test)) * 100
print(f"=> Weighted KNN (K={best_k}): {acc_weighted:.2f}%")
print(f"{'='*55}")


Tối ưu hóa K và thử Weighted KNN...

  * K= 1:  96.03%
  * K= 3:  94.90%
  * K= 5:  95.33%
  * K= 7:  94.92%
  * K= 9:  94.30%
  * K=11:  94.12%

=> K tốt nhất (uniform): K=1 → 96.03%
=> Weighted KNN (K=1): 96.03%


---
# <center>**END**<center>